# Task #1 Returning​ ​Customers #

There is a ​dataset​ ​[​cohort_data.csv​]​ ​which​ ​includes​ ​a​ ​sample​ ​of​ ​anonymised bookings​ ​from​ ​the​ ​tour booking​ ​platform​ ​since​ ​2014 with ​8,000​ ​observations.​


There are following fields in the given dataset:

* __enquiry_id__ - unique​ ​identifier​ ​of​ ​the​ ​booking
* __customer_id__ - unique​ ​customer​ ​identifier location country​ ​location​ ​of​ ​the​ ​customer 
* __booking_date__ - the​ ​date​ ​on​ ​which​ ​the​ ​booking​ ​was​ ​confirmed (these​ ​are​ ​all​ ​confirmed​ ​bookings)
* __departure_date__ - the​ ​date​ ​on​ ​which​ ​the​ ​booked​ ​tour​ ​departed
* __operator_id__ - unique​ ​operator​ ​identifier
* __tour_destination__ - primary​ ​destination​ ​for​ ​tours​ ​which​ ​may​ ​be​ ​a country,​ ​group​ ​of​ ​countries​ ​or​ ​continent passengers number​ ​of​ ​passengers​ ​included​ ​in​ ​the​ ​booking
* __tour_length__ - the​ ​length​ ​of​ ​the​ ​booked​ ​tour​ ​in​ ​days
* __booking_value__ - an​ ​obfuscated​ ​value​ ​of​ ​the​ ​booking​ ​in​ ​currency

So let's start with analysis. First of all we import all needed libraries. We will use `pandas` to work with data as it is very powerful and is an industry standard. Also we use `pandasql` to work with dataframe using standard `SQL` queries. For visualization, we use `plotly` which produces nice graphs.

In [162]:
import pandas as pd
import pandasql as ps
import numpy as np

# import matplotlib.pylab as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

Load data and have a quick look at it. We use `enquiry_id` as data entry identifier as it is unique.

In [113]:
data = pd.read_csv("cohort_data.csv", index_col="enquiry_id")
data.head(5)

,customer_id,location,booking_date,departure_date,operator_id,tour_destination,passengers,tour_length,booking_value
enquiry_id,,,,,,,,,
1,1112,Australia,2014-01-01,2014-04-29,110,Cuba,2,15,47.7143
2,1027,Australia,2014-01-01,2014-06-29,75,Italy,5,1,15.4000
3,1007,USA,2014-01-02,2014-01-17,12,Colombia,2,9,54.2000
4,87,Australia,2014-01-03,2014-07-23,11,Europe,3,15,90.0000
5,527,USA,2014-01-03,2014-09-11,47,Europe,2,18,86.4000


Convert date to standart datetime format in order to be able compare and perform operations on that data.

In [94]:
data["booking_date"] = data["booking_date"].fillna("2014-01-01").astype("datetime64[ns]")
data["departure_date"] = data["departure_date"].fillna("2014-01-01").astype("datetime64[ns]")
data.dtypes

customer_id                  int64
location                    object
booking_date        datetime64[ns]
departure_date      datetime64[ns]
operator_id                  int64
tour_destination            object
passengers                   int64
tour_length                  int64
booking_value              float64
dtype: object

How many customers come back and how often.

In [161]:
query = "SELECT customer_id, booking_date, count(*) as cnt FROM \
        data GROUP BY customer_id ORDER BY cnt DESC"
frequent_bookers = ps.sqldf(query, locals())
frequent_bookers.head()

,customer_id,booking_date,cnt
0,3891,2016-11-25,7
1,204,2016-08-16,5
2,2204,2015-11-02,5
3,2642,2016-05-28,5
4,505,2016-09-23,4


In [159]:
iplot(go.Figure(
    data=[go.Histogram(x=frequent_bookers["cnt"])], 
    layout=go.Layout(title="Returning customers",
                     yaxis=dict(type='log', title="Number of customers"),
                     xaxis=dict(title="Number of bookings by the customer"))))

In [147]:
returning_customers = data["customer_id"].value_counts()[data["customer_id"].value_counts()>1]
number_of_returning = len(returning_customers) / len(data["customer_id"].value_counts())
print("Customers who booked only once: %d or %.2f" %
     (len(data["customer_id"].value_counts() - (len(returning_customers))),
      (1-number_of_returning) * 100) + "%")
print("Returning customers: %d or %.2f" % (len(returning_customers), number_of_returning * 100) + "%")

Customers who booked only once: 7438 or 93.57%
Returning customers: 478 or 6.43%


We see that most of the customers don't return and only ~6.5% book 2 or more times, with one loyal customer who used TourRadar 7 times!

In [163]:
query = "select customer_id, max(julianday(booking_date))-min(julianday(booking_date)) as spread \
from data \
group by customer_id \
order by spread desc, customer_id"
top = ps.sqldf(query, locals())
iplot(go.Figure(
    data=[go.Histogram(x=top["spread"], xbins=dict(start=1,end=1000,size=50))], 
    layout=go.Layout(title="Maximum spread between bookings by one customer",
                     yaxis=dict(title="Number of returning customers"),
                     xaxis=dict(title="Maximum difference between bookings in days"))))